In [1]:
!pip install q keras==1.2.2
!pip install tensorflow==1.4

In [3]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [4]:
import os
os.chdir('./drive/My Drive/TESI/ST-ResNet')

In [12]:
# from __future__ import print_function
# import os
# import sys
# import pickle
import time
# import numpy as np
import h5py

# from keras.optimizers import Adam
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# from deepst.models.STResNet import stresnet
# from deepst.config import Config
# import deepst.metrics as metrics
# from deepst.datasets import TaxiBJ

from __future__ import print_function
import os
# import cPickle as pickle
import pickle
import numpy as np
import math

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from deepst.models.STResNet import stresnet
from deepst.config import Config
import deepst.metrics as metrics
from deepst.datasets import BikeNYC

In [6]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

1.2.2
1.4.0


In [14]:
np.random.seed(1337)  # for reproducibility

# parameters
DATAPATH = '../data'  # data path, you may set your own data path with the global envirmental variable DATAPATH
CACHEDATA = True  # cache data or NOT
path_cache = os.path.join(DATAPATH, 'CACHE')  # cache path
nb_epoch = 500  # number of epoch at training stage
nb_epoch_cont = 100  # number of epoch at training (cont) stage
batch_size = 32  # batch size

T = 24  # number of time intervals in one day
lr = 0.0002  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence
nb_residual_unit = 4 # paper says 4 for BikeNYC

nb_flow = 2  # there are two types of flows: inflow and outflow
# divide data into two subsets: Train & Test, of which the test set is the
# last 4 weeks
# last 10 days
days_test = 10
len_test = T * days_test
map_height, map_width = 16, 8  # grid size
# For NYC Bike data, there are 81 available grid-based areas, each of
# which includes at least ONE bike station. Therefore, we modify the final
# RMSE by multiplying the following factor (i.e., factor).
nb_area = 81
m_factor = math.sqrt(1. * map_height * map_width / nb_area)
print('factor: ', m_factor)
path_result = 'RET'
path_model = 'MODEL'

factor:  1.2570787221094177


In [ ]:
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [7]:
def build_model(external_dim):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None
    model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model

In [9]:
def read_cache(fname):
    mmn = pickle.load(open('preprocessing.pkl', 'rb'))

    f = h5py.File(fname, 'r')
    num = int(f['num'].value)
    X_train, Y_train, X_test, Y_test = [], [], [], []
    for i in range(num):
        X_train.append(f['X_train_%i' % i].value)
        X_test.append(f['X_test_%i' % i].value)
    Y_train = f['Y_train'].value
    Y_test = f['Y_test'].value
    external_dim = f['external_dim'].value
    timestamp_train = f['T_train'].value
    timestamp_test = f['T_test'].value
    f.close()

    return X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test

In [10]:
def cache(fname, X_train, Y_train, X_test, Y_test, external_dim, timestamp_train, timestamp_test):
    h5 = h5py.File(fname, 'w')
    h5.create_dataset('num', data=len(X_train))

    for i, data in enumerate(X_train):
        h5.create_dataset('X_train_%i' % i, data=data)
    # for i, data in enumerate(Y_train):
    for i, data in enumerate(X_test):
        h5.create_dataset('X_test_%i' % i, data=data)
    h5.create_dataset('Y_train', data=Y_train)
    h5.create_dataset('Y_test', data=Y_test)
    external_dim = -1 if external_dim is None else int(external_dim)
    h5.create_dataset('external_dim', data=external_dim)
    h5.create_dataset('T_train', data=timestamp_train)
    h5.create_dataset('T_test', data=timestamp_test)
    h5.close()

In [15]:
print("loading data...")
ts = time.time()
fname = os.path.join(DATAPATH, 'CACHE', 'BikeNYC_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = read_cache(
        fname)
    print("load %s successfully" % fname)
else:
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = BikeNYC.load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        preprocess_name='preprocessing.pkl', meta_data=True, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train, Y_train, X_test, Y_test,
              external_dim, timestamp_train, timestamp_test)

print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])
print("\nelapsed time (loading data): %.3f seconds\n" % (time.time() - ts))

print('=' * 10)


loading data...


/content/drive/My Drive/TESI/ST-ResNet/deepst/datasets/__init__.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  data = f['data'].value
/content/drive/My Drive/TESI/ST-ResNet/deepst/datasets/__init__.py:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  timestamps = f['date'].value


incomplete days:  []
train_data shape:  (4152, 2, 16, 8)
min: 0.0 max: 267.0
XC shape:  (3720, 6, 16, 8) XP shape:  (3720, 8, 16, 8) XT shape:  (3720, 8, 16, 8) Y shape: (3720, 2, 16, 8)
XC shape:  (3720, 6, 16, 8) XP shape:  (3720, 8, 16, 8) XT shape:  (3720, 8, 16, 8) Y shape: (3720, 2, 16, 8)
train shape: (3480, 6, 16, 8) (3480, 2, 16, 8) test shape:  (240, 6, 16, 8) (240, 2, 16, 8)
(3480, 6, 16, 8)
(3480, 8, 16, 8)
(3480, 8, 16, 8)
(3480, 8)

(240, 6, 16, 8)
(240, 8, 16, 8)
(240, 8, 16, 8)
(240, 8)


 days (test):  [b'20140921', b'20140922', b'20140923', b'20140924', b'20140925', b'20140926', b'20140927', b'20140928', b'20140929', b'20140930']

elapsed time (loading data): 43.600 seconds



In [16]:
#from deepst.datasets import stat; stat('BJ16_M32x32_T30_InOut.h5')
f = h5py.File('../data/BikeNYC/NYC14_M16x8_T60_NewEnd.h5', 'r')
print(len(f['date']))
# print(f['data'][1][0][0])

print(len(X_train))
print(len(X_train[0]))
print(len(X_train[0][0]))
xtrain_np = np.array(X_train[0], dtype=np.float32)
print(xtrain_np.shape)
print(len(Y_train))
ytrain_np = np.array(Y_train, dtype=np.float32)
print(ytrain_np.shape)
print(len(timestamp_train))
print(len(timestamp_test))
print(len(X_test))

4392
4
3480
6
(3480, 6, 16, 8)
3480
(3480, 2, 16, 8)
3480
240
4


In [17]:
print("compiling model...")
print(
    "**at the first time, it takes a few minites to compile if you use [Theano] as the backend**")
ts = time.time()
model = build_model(external_dim)
hyperparams_name = 'BikeNYC.c{}.p{}.t{}.resunit{}.lr{}'.format(
    len_closeness, len_period, len_trend, nb_residual_unit, lr)
fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

early_stopping = EarlyStopping(monitor='val_rmse', patience=2, mode='min')
model_checkpoint = ModelCheckpoint(
    fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

print("\nelapsed time (compiling model): %.3f seconds\n" %
      (time.time() - ts))

print('=' * 10)

compiling model...
**at the first time, it takes a few minites to compile if you use [Theano] as the backend**
(?, 2, 16, 8)
(?, 2, 16, 8)
(?, 2, 16, 8)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 6, 16, 8)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 8, 16, 8)      0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 8, 16, 8)      0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None,

In [ ]:
print("training model...")
ts = time.time()
history = model.fit(X_train, Y_train,
                    nb_epoch=nb_epoch,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[early_stopping, model_checkpoint],
                    verbose=1)
model.save_weights(os.path.join(
    'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
pickle.dump((history.history), open(os.path.join(
    path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))
